In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [24]:
poet_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world-renowned poet known for your deep understanding of programming languages. Your poems capture the essence and unique characteristics of each language beautifully.",
        ),
        ("human", "Write a poem about the {language} programming language."),
    ]
)

poet_chain = poet_prompt | chat

In [25]:
explainer_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert in poetry analysis. Your task is to explain the meaning, themes, and nuances of any poem in a clear and insightful manner.",
        ),
        ("human", "Explain the following poem:\n\n{poem}"),
    ]
)

explainer_chain = explainer_prompt | chat

{'language': 'python',
 'poem': "In the land of code, where languages reside,\nThere dwells a serpent, sleek and wise.\nPython is its name, a language renowned,\nFor its simplicity and power unbound.\n\nWith syntax clean and easy to read,\nIt slithers through tasks with lightning speed.\nFrom web development to data science's art,\nPython weaves its magic in every part.\n\nIts libraries vast, its capabilities vast,\nPython shines bright, unsurpassed.\nFrom machine learning to automation's hand,\nPython reigns supreme, a language grand.\n\nSo let us embrace this serpent of code,\nAnd unleash its power in every abode.\nFor with Python by our side, we can soar,\nAnd conquer the digital world like never before.",
 'explanation': 'This poem describes Python, a popular programming language known for its simplicity, readability, and versatility. The poem personifies Python as a sleek and wise serpent that navigates through tasks quickly and efficiently.\n\nPython is praised for its clean synt

In [ ]:
final_chain = {"poem": poet_chain} | explainer_chain

final_chain.invoke({"language": "python"})